In [2]:
import os, pdb, json, re, keras, vis_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential, load_model, Model, model_from_json
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, InputLayer
from keras.utils import np_utils, generic_utils
from keras import backend as K
from PIL import Image
import numpy as np
from tqdm import tnrange, tqdm_notebook
import gc
import time
import tensorflow as tf
sess = tf.Session()
import matplotlib.pyplot as plt
%matplotlib inline
K.set_session(sess)

/home/icarus/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
model_json = vis_utils.json_load('models.json')

In [4]:
model_type = 'cifar'
top_level= [_dir for _dir in os.listdir(model_type) if not _dir.startswith('.')]
dir_paths = {entry:os.path.join(model_type, entry) for entry in top_level}
jsons = os.listdir(dir_paths['jsons'])
model_dirs = [os.path.splitext(_item)[0] for _item in jsons]
train_data_dir = 'data/' + model_type + '/train'
test_data_dir = 'data/' + model_type + '/test'
class_name = {i:str(model_json[model_type]['labels'][i]) for i in range(len(model_json[model_type]['labels']))}

In [5]:
len(class_name)

10

In [18]:
#for idx in tqdm_notebook(range(len(jsons)), desc="Models"):
for idx in tqdm_notebook(range(1,6), desc="Models"):
#idx = 0
#print jsons[idx]
    dir_list = vis_utils.dir_list_returner(dir_paths, ['weights','activations','supplement'], model_dirs[idx])
    #Set up supplemental parameters
    compile_params = vis_utils.json_load(os.path.join(dir_list['supplement'], 'compile.json'))
    # Loading the Model file
    #with open(os.path.join(dir_paths['jsons'],jsons[idx]), 'r') as model_file:
    #    model = model_from_json(model_file.read())
    #compile_params = vis_utils.json_load(os.path.join(dir_list['supplement'], 'compile.json'))
    #model.compile(loss = compile_params['loss'], optimizer = compile_params['optimizer'], metrics=compile_params['metrics'])
    weight_files = [_file for _file in os.listdir(dir_list['weights']) if _file.startswith('weights')]
    weight_files.sort()
    for wt_idx in tqdm_notebook([14,9,4,0], desc = jsons[idx], leave=False):
    #wt_idx = 14
        weight_folder = os.path.join(dir_list['activations'], 'epoch'+str(wt_idx))
        make = os.makedirs(weight_folder) if not os.path.exists(weight_folder) else None
        model=load_model(os.path.join(dir_list['weights'], weight_files[wt_idx]))
        model.trainable = False
        model.compile(loss = compile_params['loss'], optimizer = compile_params['optimizer'], metrics=compile_params['metrics'])
        conv_layers = vis_utils.get_conv_layers(model)
        layer_activations = K.function([model.layers[0].input, K.learning_phase()],
                                          [layer.output for layer in model.layers if layer.name.startswith('conv')])
        for idx,_class in enumerate(tqdm_notebook(class_name, desc="Class", leave=False)):
            #_idx = 0
            #_class = class_name[_idx]
            class_target = os.path.join(weight_folder,class_name[_class])

            make = os.makedirs(class_target) if not os.path.exists(class_target) else None
            imgs = os.listdir(os.path.join(test_data_dir, class_name[_class]))
            for _image in tqdm_notebook(imgs, desc="Image", leave=False):
            #_image = imgs[32]
                ker_img= np.array([np.array(Image.open(os.path.join(test_data_dir, class_name[_class]) + '/'+_image)).astype("float32")/255])
                target = os.path.join(class_target, os.path.splitext(_image)[0])
                #res=model.predict(ker_img)
                #soft_res = np.exp(res)/np.sum(np.exp(res))
                #if int(np.where(soft_res == np.max(soft_res))[1]) != idx <---- if incorrect
                act = layer_activations([ker_img, 0])
                act_means = [np.mean(_act, axis=(0,1,2)) for _act in act]
                indices = [np.argsort(_means) for _means in act_means]
                activation_dict = {layer:{'sorted':indices[idx].tolist(), 'activations':act_means[idx].tolist()} for idx, layer in enumerate(conv_layers)}
                with open(target, 'w') as _compile:
                        json.dump(activation_dict, _compile)

In [8]:
load_model(os.path.join(dir_list['weights'], weight_files[wt_idx]))

In [12]:
class_name

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}